In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
df_train_est_A = pd.read_csv('train_est_B.csv')
df_train_obs_A = pd.read_csv('train_obs_B.csv')
df_test_A = pd.read_csv('X_test_B.csv')
df_train_est_A = df_train_est_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_train_obs_A = df_train_obs_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_test_A = df_test_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_train_est_A.set_index('date_forecast', inplace=True)
df_train_obs_A.set_index('date_forecast', inplace=True)
df_test_A.set_index('date_forecast', inplace=True)
df_train_obs_A.index = pd.to_datetime(df_train_obs_A.index)
df_train_obs_A.index = pd.to_datetime(df_train_obs_A.index)

In [23]:
df = pd.concat([df_train_obs_A,df_train_est_A])
df = df.drop(columns = 'date_calc', axis = 1)
df_test_A = df_test_A.drop(columns = 'date_calc', axis = 1)

In [24]:
df.isna().sum(), df.shape

(absolute_humidity_2m:gm3             24
 air_density_2m:kgm3                  24
 ceiling_height_agl:m               5183
 clear_sky_energy_1h:J                24
 clear_sky_rad:W                      24
 cloud_base_agl:m                   2196
 dew_or_rime:idx                      24
 dew_point_2m:K                       24
 diffuse_rad:W                        24
 diffuse_rad_1h:J                     24
 direct_rad:W                         24
 direct_rad_1h:J                      24
 effective_cloud_cover:p              24
 elevation:m                          24
 fresh_snow_12h:cm                    24
 fresh_snow_1h:cm                     24
 fresh_snow_24h:cm                    24
 fresh_snow_3h:cm                     24
 fresh_snow_6h:cm                     24
 is_day:idx                           24
 is_in_shadow:idx                     24
 msl_pressure:hPa                     24
 precip_5min:mm                       24
 precip_type_5min:idx                 24
 pressure_100m:h

In [25]:
df['ceiling_height_agl:m'].describe()

count    27664.000000
mean      2735.589765
std       2425.728099
min         27.849998
25%       1065.837500
50%       1806.575000
75%       3627.556250
max      12285.775000
Name: ceiling_height_agl:m, dtype: float64

In [26]:
df['cloud_base_agl:m'].describe()

count    30651.000000
mean      1728.566279
std       1730.413540
min         27.975000
25%        636.800000
50%       1204.000000
75%       2075.412500
max      11673.725000
Name: cloud_base_agl:m, dtype: float64

250 means settled snow

In [27]:
df['snow_density:kgm3'].describe()

count    1606.0
mean      250.0
std         0.0
min       250.0
25%       250.0
50%       250.0
75%       250.0
max       250.0
Name: snow_density:kgm3, dtype: float64

In [28]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [29]:
df_nandrop = df
df_nandrop = df_nandrop.drop(columns= ['snow_densitykgm3'],axis = 1)
df_nandrop = df_nandrop.dropna()

df_nan0 = df.fillna(0)
df_nanmedian = df.fillna(df.median())

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
# Imputa i valori mancanti
df_knn =pd.DataFrame(imputer.fit_transform(df),columns = df.columns, index = df.index)

In [37]:
df_dict = {
    'drop' : df_nandrop,
    'zero' : df_nan0,
    'median' : df_nanmedian,
    'knn' : df_knn,
}

In [38]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error
# Define the number of splits (e.g., 5 for 80-20 train-test splits)
n_splits = 3
# Initialize the TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

In [39]:
import lightgbm as lgb
param = {'verbose' :-1}

model = lgb.LGBMRegressor(**param)

In [40]:
def test(df_dict):
    mean_mae_list = []
    for key in df_dict:
        print(key)
        data = df_dict[key]
        maelist = []
        for train_index, test_index in tscv.split(data):
            train_data = data.iloc[train_index]
            test_data = data.iloc[test_index]

            # Extract target variable for training and testing data
            y_train = train_data['pv_measurement']
            y_test = test_data['pv_measurement']

            # Extract features for training and testing data
            X_train = train_data.drop(columns = 'pv_measurement')
            X_test = test_data.drop(columns = 'pv_measurement')

            # Train the XGBoost model
            model.fit(X_train, y_train,eval_set=[(X_test,y_test)])

            # Make predictions on the test data
            y_pred = model.predict(X_test)

            # Evaluate the model using Mean Absolute Error (MAE)
            mae = mean_absolute_error(y_test, y_pred)
            print("Mean Absolute Error:", mae)
            maelist.append(mae)
        # Return MAE
        mean_mae = np.average(maelist,weights=[1,2,3])
        print("Mean MAE: ",mean_mae )
        mean_mae_list.append(mean_mae)
    return mean_mae_list

In [41]:
test(df_dict)

drop
Mean Absolute Error: 58.1216513495905
Mean Absolute Error: 58.79769498360033
Mean Absolute Error: 21.260812865288816
Mean MAE:  39.91657998544293
zero
Mean Absolute Error: 61.666629114794375
Mean Absolute Error: 61.64201403120569
Mean Absolute Error: 25.271109835252506
Mean MAE:  43.460664447160546
median
Mean Absolute Error: 61.19238067397233
Mean Absolute Error: 62.176562632539635
Mean Absolute Error: 24.971669782797626
Mean MAE:  43.41008588124075
knn
Mean Absolute Error: 61.8435176692509
Mean Absolute Error: 62.070886175829806
Mean Absolute Error: 24.777432110880927
Mean MAE:  43.38626439225888


[39.91657998544293, 43.460664447160546, 43.41008588124075, 43.38626439225888]

In [42]:
columns = df_nandrop.drop(columns = 'pv_measurement').columns


In [46]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Crea un oggetto MinMaxScaler
scaler = MinMaxScaler()

# Normalizza il DataFrame
df_normalized = pd.DataFrame(scaler.fit_transform(df_nandrop[columns]),columns = df_nandrop[columns].columns, index = df_nandrop[columns].index)
df_normalized['pv_measurement'] = df_nandrop[ 'pv_measurement']

In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Crea un oggetto StandardScaler
scaler = StandardScaler()

# Standardizza il DataFrame
df_scaled = pd.DataFrame(scaler.fit_transform(df_nandrop[columns]),columns = df_nandrop[columns].columns, index = df_nandrop[columns].index)
df_scaled['pv_measurement'] = df_nandrop[ 'pv_measurement']

In [48]:
def z_score_standardization(data):
  mean = np.mean(data)
  std = np.std(data)
  standardized_data = (data - mean) / std
  return standardized_data

def z_score_standardization_dataframe(df):
    standardized_df = pd.DataFrame()
    for column in df.columns:
        standardized_df[column] = z_score_standardization(df[column])
    return standardized_df

# Esempio di utilizzo della funzione:

df_zscore = df_nandrop[columns]

df_zscore = z_score_standardization_dataframe(df_zscore)
df_zscore['pv_measurement'] = df_nandrop[ 'pv_measurement']

In [49]:
df_scaled_norm = {
    'norm' : df_normalized,
    'scaled' : df_scaled,
    'zscore' : df_zscore,
}

In [50]:
test(df_scaled_norm)

norm
Mean Absolute Error: 57.50500452677472
Mean Absolute Error: 58.799405865373
Mean Absolute Error: 21.463545639987125
Mean MAE:  39.91574219624702
scaled
Mean Absolute Error: 57.745511156238265
Mean Absolute Error: 59.48216270329596
Mean Absolute Error: 21.435341798558014
Mean MAE:  40.16931032641737
zscore
Mean Absolute Error: 57.74498542323928
Mean Absolute Error: 59.48216270329596
Mean Absolute Error: 21.435341798558014
Mean MAE:  40.16922270425087


[39.91574219624702, 40.16931032641737, 40.16922270425087]

In [ ]:
df_normalized['direct_radW'].plot()

In [ ]:
df_scaled['direct_radW'].plot()

In [ ]:
df_zscore['direct_radW'].plot()